In [88]:
import torch
# a=torch.tensor([[1,2,3],[2,3,4]])
# print((a.shape))
# amax=a.max(-1)
# asum=a.sum(-1)
# asum=asum.unsqueeze(-1)
# a-asum

x=torch.rand(16,6,197,768)
# testMax=x.max(-1)
# print(testMax.values.shape)
# Row_Sum=(torch.pow(2,x-x.max(-1).values.unsqueeze(-1))).sum(-1)#先算一下每一行的指数累加和作为分母
# print(Row_Sum.unsqueeze(-1).shape)
# # print(Row_Sum.unsqueeze(-1).expand(16,6,197,768))
# Row_Sum=Row_Sum.unsqueeze(-1).expand(16,6,197,768)
# Row_Sum.shape[0:3]

Row_Sum=(torch.pow(2,x-x.max(-1).values.unsqueeze(-1))).sum(-1)#先算一下每一行的指数累加和作为分母
Row_Sum=Row_Sum.unsqueeze(-1)
Row_Sum=Row_Sum.expand(Row_Sum.shape[0],Row_Sum.shape[1],Row_Sum.shape[2],x.shape[-1])
x=torch.pow(2,x-x.max(-1).values.unsqueeze(-1))/Row_Sum

In [64]:
x = torch.Tensor([[[1], [2], [3]]])
print(x.expand(1,3, 5))


tensor([[[1., 1., 1., 1., 1.],
         [2., 2., 2., 2., 2.],
         [3., 3., 3., 3., 3.]]])


In [1]:
import torch
a=torch.arange(12)
print(a)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])


In [9]:
import torch
a=torch.arange(12).reshape(3,2,2).flatten(1)
print(a)#按第1个维度展开，你应该能理解，就是把这个维度的所有元素都展开
a=torch.arange(12).reshape(3,2,2).flatten(2)
a#可以认为按最后一个维度展开不会有任何改变

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])


tensor([[[ 0,  1],
         [ 2,  3]],

        [[ 4,  5],
         [ 6,  7]],

        [[ 8,  9],
         [10, 11]]])

In [7]:

import torch
from torchvision.models import AlexNet
import torch.nn as nn
from torchviz import make_dot

blocks=nn.Transformer(d_model=768,nhead=12, num_encoder_layers=1,num_decoder_layers=1)
print(blocks)
x=torch.rand(1,197,768)
out=blocks(x,x)

g = make_dot(out)

Transformer(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (linear1): Linear(in_features=768, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=768, bias=True)
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
    (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (decoder): TransformerDecoder(
    (layers): ModuleList(
      (0): TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=

In [ ]:
import torch
from torchvision.models import AlexNet
import torch.nn as nn
from torchviz import make_dot
from collections import OrderedDict
from functools import partial
from itertools import repeat
import collections.abc
#Trasnforemr相关组件====================================
def _ntuple(n):
    def parse(x):
        if isinstance(x, collections.abc.Iterable):
            return x
        return tuple(repeat(x, n))

    return parse
to_2tuple = _ntuple(2)

class PatchEmbed(nn.Module):#默认为vit—base
    # patch_size=16,
    # embed_dim=768,
    # depth=12,
    # num_heads=12,
    # mlp_ratio=4,

    def __init__(self,In_Channels:int=1,Out_Channels:int=768,img_size:int=224,patch_size:int=16):
        super(PatchEmbed,self).__init__()
        self.proj=nn.Conv2d(in_channels=In_Channels,out_channels=Out_Channels, kernel_size=patch_size,stride=patch_size,)
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.grid_size = (img_size[0] // patch_size[0],
                          img_size[1] // patch_size[1])
        self.num_patches = self.grid_size[0] * self.grid_size[1]
    def forward(self,x):
        print(x.shape)
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x
def Test_PatchEmbed():
    model=PatchEmbed(In_Channels=1,Out_Channels=384)
    model(torch.rand(1,1,224,224))


In [ ]:
x=torch.rand(2,3,224,224)
Embedding_Layer=PatchEmbed(3,768,224,224)
Single_Encoder = nn.TransformerEncoderLayer(d_model=768, nhead=8,batch_first=True,norm_first=True)#在Vit中的norm需要位于atten和FF之前
Encoders= nn.TransformerEncoder(Single_Encoder,num_layers=1,norm=None)#构建多个连在一起的Encoder
                                                                      #nomrm 就是在末尾加一个normlization

x=Embedding_Layer(x)
print(x.shape)
x=Encoders(x)                                                 

print(x.shape)
print(Single_Encoder)

In [1]:
for i in(1,5):
    print(i)

1
5
